## The purpose of this notebook is to turn the datasets returned from the lag creator to be grouped by the day or hour and add the features extracted for that day or hour (average polarity and volume)

In [2]:
#imports
import pandas as pd
import os

In [3]:
#get data with following lag
granularity = "days"
lag = 3
# analyser = "TextBlob"
analyser = "vader"

#how to group - by "day" or "hour"
group_by = "day"

In [3]:
#prepare filename to read
folder = "./../datasets/tweets_and_prices/"+analyser+"/";
filename = folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"

In [4]:
#read data
df = pd.read_csv(filename)

C:\Users\Lenovo T440s i7\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.head()

,Unnamed: 0,Unnamed: 0.1,id,user,fullname,url,timestamp,replies,likes,retweets,text,compound,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,0,0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:00+00:00,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.0000,8696.42,0.244451,2126.830315,8700.44397
1,1,1,1.132977e+18,bitcointe,Bitcointe,NaN,2019-05-27 11:49:00+00:00,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,-0.1027,8696.42,0.244451,2126.830315,8700.44397
2,2,2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:00+00:00,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.0000,8696.42,0.244451,2126.830315,8700.44397
3,3,3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:00+00:00,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.0000,8696.42,0.244451,2126.830315,8700.44397
4,4,4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:00+00:00,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.3612,8696.42,0.244451,2126.830315,8700.44397


In [6]:
#remove extra columns
df = df.drop(['Unnamed: 0'], axis=1)

In [7]:
#get datetime
df["dt"] = pd.to_datetime(df['timestamp'])
df = df.drop(['timestamp'], axis=1)

In [8]:
#copy dataframe so that we do not have to stay reloading data after changes
df_copy = df.copy()

In [9]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14941649 entries, 0 to 14941648
Data columns (total 15 columns):
 #   Column             Dtype              
---  ------             -----              
 0   Unnamed: 0.1       int64              
 1   id                 float64            
 2   user               object             
 3   fullname           object             
 4   url                object             
 5   replies            float64            
 6   likes              float64            
 7   retweets           float64            
 8   text               object             
 9   compound           float64            
 10  Close              float64            
 11  Volume_(BTC)       float64            
 12  Volume_(Currency)  float64            
 13  Weighted_Price     float64            
 14  dt                 datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(9), int64(1), object(4)
memory usage: 1.7+ GB


In [10]:
df_copy.head()

,Unnamed: 0.1,id,user,fullname,url,replies,likes,retweets,text,compound,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,dt
0,0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00
1,1,1.132977e+18,bitcointe,Bitcointe,NaN,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,-0.1027,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00
2,2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00
3,3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00
4,4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.3612,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00


In [11]:
#floor datetimes to day or hour
df_copy['DateTime'] = df_copy['dt'].dt.floor(group_by[0])

In [12]:
df_copy.head()

,Unnamed: 0.1,id,user,fullname,url,replies,likes,retweets,text,compound,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,dt,DateTime
0,0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
1,1,1.132977e+18,bitcointe,Bitcointe,NaN,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,-0.1027,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
2,2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
3,3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
4,4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.3612,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00


In [13]:
df_copy['DateTime']

0          2019-05-27 00:00:00+00:00
1          2019-05-27 00:00:00+00:00
2          2019-05-27 00:00:00+00:00
3          2019-05-27 00:00:00+00:00
4          2019-05-27 00:00:00+00:00
                      ...           
14941644   2019-11-23 00:00:00+00:00
14941645   2019-11-23 00:00:00+00:00
14941646   2019-11-23 00:00:00+00:00
14941647   2019-11-23 00:00:00+00:00
14941648   2019-11-23 00:00:00+00:00
Name: DateTime, Length: 14941649, dtype: datetime64[ns, UTC]

In [14]:
#group tweet volumes daily
grouped_volumes = df_copy.groupby(pd.Grouper(key='dt',freq=group_by[0])).size().reset_index(name='tweet_vol')

In [15]:
#set index to datetime
grouped_volumes.index = pd.to_datetime(grouped_volumes.index)
grouped_volumes = grouped_volumes.set_index('dt')

In [16]:
#set type of volumes to float
grouped_volumes['tweet_vol'] = grouped_volumes['tweet_vol'].astype(float)

In [17]:
grouped_volumes.head()

,tweet_vol
dt,
2018-08-28 00:00:00+00:00,548.0
2018-08-29 00:00:00+00:00,971.0
2018-08-30 00:00:00+00:00,950.0
2018-08-31 00:00:00+00:00,904.0
2018-09-01 00:00:00+00:00,871.0


In [18]:
#set datetime to index of dataframe
df_copy.index = pd.to_datetime(df_copy.index)

In [19]:
#group by day and take the average of values in that day
df_copy = df_copy.groupby('DateTime').agg(lambda x: x.mean())

In [20]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 453 entries, 2018-08-28 00:00:00+00:00 to 2019-11-23 00:00:00+00:00
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Unnamed: 0.1       453 non-null    float64            
 1   id                 453 non-null    float64            
 2   replies            453 non-null    float64            
 3   likes              453 non-null    float64            
 4   retweets           453 non-null    float64            
 5   compound           453 non-null    float64            
 6   Close              453 non-null    float64            
 7   Volume_(BTC)       453 non-null    float64            
 8   Volume_(Currency)  453 non-null    float64            
 9   Weighted_Price     453 non-null    float64            
 10  dt                 453 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(10)
memory usage: 42.5 KB


In [21]:
df.head()

,Unnamed: 0.1,id,user,fullname,url,replies,likes,retweets,text,compound,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,dt
0,0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00
1,1,1.132977e+18,bitcointe,Bitcointe,NaN,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,-0.1027,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00
2,2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00
3,3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00
4,4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.3612,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00


In [22]:
#set DateTime 
df['DateTime'] = df['dt'].dt.floor(group_by[0])

In [23]:
df.head()

,Unnamed: 0.1,id,user,fullname,url,replies,likes,retweets,text,compound,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,dt,DateTime
0,0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
1,1,1.132977e+18,bitcointe,Bitcointe,NaN,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,-0.1027,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
2,2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
3,3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.0000,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
4,4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.3612,8696.42,0.244451,2126.830315,8700.44397,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00


In [24]:
#get final prices of day
df_copy["Close"] = pd.Series(df.loc[df.groupby('DateTime').dt.idxmax()]["Close"]).values
# df_copy["Open"] = pd.Series(df.loc[df.groupby('DateTime').dt.idxmin()]["Open"]).values
# df_copy["High"] = pd.Series(df.loc[df.groupby('DateTime').Close.idxmax()]["High"]).values
# df_copy["Low"] = pd.Series(df.loc[df.groupby('DateTime').Close.idxmin()]["Low"]).values

In [25]:
df_copy.head()

,Unnamed: 0.1,id,replies,likes,retweets,compound,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,dt
DateTime,,,,,,,,,,,
2018-08-28 00:00:00+00:00,3.198982e+06,1.034519e+18,9.468978,12.708029,7.416058,0.047422,7027.99,11.046760,76827.202556,6978.965226,2018-08-28 17:20:24.525543168+00:00
2018-08-29 00:00:00+00:00,2.936068e+06,1.034807e+18,1.049434,11.010299,5.277034,0.044971,7187.23,8.339695,59615.092302,7094.262473,2018-08-29 12:21:55.983516672+00:00
2018-08-30 00:00:00+00:00,3.130871e+06,1.035163e+18,1.090526,8.868421,3.075789,0.045871,7275.00,2.340197,16923.909967,7232.208998,2018-08-30 11:58:35.305266176+00:00
2018-08-31 00:00:00+00:00,2.915478e+06,1.035525e+18,0.357301,8.348451,3.175885,0.037400,7301.01,2.340134,16985.523253,7260.615036,2018-08-31 11:51:55.154861312+00:00
2018-09-01 00:00:00+00:00,2.819986e+06,1.035885e+18,0.874856,5.466131,1.732491,0.033980,7350.00,3.563551,26021.887952,7315.184597,2018-09-01 11:42:32.307692288+00:00


In [26]:
#add tweet volume to df
df_copy['Tweet_vol'] = grouped_volumes['tweet_vol']

In [27]:
df_copy.head()

,Unnamed: 0.1,id,replies,likes,retweets,compound,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,dt,Tweet_vol
DateTime,,,,,,,,,,,,
2018-08-28 00:00:00+00:00,3.198982e+06,1.034519e+18,9.468978,12.708029,7.416058,0.047422,7027.99,11.046760,76827.202556,6978.965226,2018-08-28 17:20:24.525543168+00:00,548.0
2018-08-29 00:00:00+00:00,2.936068e+06,1.034807e+18,1.049434,11.010299,5.277034,0.044971,7187.23,8.339695,59615.092302,7094.262473,2018-08-29 12:21:55.983516672+00:00,971.0
2018-08-30 00:00:00+00:00,3.130871e+06,1.035163e+18,1.090526,8.868421,3.075789,0.045871,7275.00,2.340197,16923.909967,7232.208998,2018-08-30 11:58:35.305266176+00:00,950.0
2018-08-31 00:00:00+00:00,2.915478e+06,1.035525e+18,0.357301,8.348451,3.175885,0.037400,7301.01,2.340134,16985.523253,7260.615036,2018-08-31 11:51:55.154861312+00:00,904.0
2018-09-01 00:00:00+00:00,2.819986e+06,1.035885e+18,0.874856,5.466131,1.732491,0.033980,7350.00,3.563551,26021.887952,7315.184597,2018-09-01 11:42:32.307692288+00:00,871.0


In [28]:
#remove tweet id, retweets, likes, replies and currency volume
df_copy = df_copy.drop(['id', 'retweets', 'likes', 'replies', 'Volume_(Currency)'], axis=1)

In [29]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 453 entries, 2018-08-28 00:00:00+00:00 to 2019-11-23 00:00:00+00:00
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   Unnamed: 0.1    453 non-null    float64            
 1   compound        453 non-null    float64            
 2   Close           453 non-null    float64            
 3   Volume_(BTC)    453 non-null    float64            
 4   Weighted_Price  453 non-null    float64            
 5   dt              453 non-null    datetime64[ns, UTC]
 6   Tweet_vol       453 non-null    float64            
dtypes: datetime64[ns, UTC](1), float64(6)
memory usage: 28.3 KB


In [30]:
folder = "./../datasets/tweets_prices_volumes_sentiment/"+analyser+"/"+group"_datasets/"

if not os.path.exists(folder):
    os.makedirs(folder)

In [31]:
filename = folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"
df_copy.to_csv(filename)